<a href="https://colab.research.google.com/github/ccsed/ProgettoLabAI/blob/main/File1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Collego colab a google drive


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Importo le librerie necessarie

In [2]:
!pip install rasterio

In [3]:
import os
import numpy as np
import rasterio
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import albumentations as A
from PIL import Image
import matplotlib.pyplot as plt
import random

Creo la funzione per dividere il dataset in train, validation e test

In [4]:
def split_dataset(image_dir, drive_dir, train_ratio=0.7, val_ratio=0.2, test_ratio=0.1, seed=42):
  images = sorted(os.listdir(image_dir))

  combined = [(img, img) for img in images]
  random.seed(seed)
  random.shuffle(combined)

  train_split = int(len(combined) * train_ratio)
  val_split = int(len(combined) * (train_ratio + val_ratio))
  train_data = combined[:train_split]
  val_data = combined[train_split:val_split]
  test_data = combined[val_split:]

  with open(os.path.join(drive_dir, 'train.txt'), 'w') as f:
    for img, lbl in train_data:
      f.write(f"{img} {lbl}\n")

  with open(os.path.join(drive_dir, 'val.txt'), 'w') as f:
    for img, lbl in val_data:
      f.write(f"{img} {lbl}\n")

  with open(os.path.join(drive_dir, 'test.txt'), 'w') as f:
    for img, lbl in test_data:
      f.write(f"{img} {lbl}\n")


Definisco la directory dove salvare i file txt e faccio split sul dataset

In [ ]:
drive_dir = '/content/drive/MyDrive'
image_dir = '/content/drive/MyDrive/Data/SN6_buildings_AOI_11_Rotterdam_train/train/AOI_11_Rotterdam/SAR-Intensity'
split_dataset(image_dir, drive_dir)

Per visualizzare un'immagine

In [ ]:
image_path = '/content/drive/MyDrive/Data/SN6_buildings_AOI_11_Rotterdam_train/train/AOI_11_Rotterdam/SAR-Intensity/SN6_Train_AOI_11_Rotterdam_SAR-Intensity_20190804113009_20190804113242_tile_4800.tif'
with rasterio.open(image_path) as src:
  image = src.read()
  print(f"Image shape: {image.shape}")  # (channels, height, width)
fig, axs = plt.subplots(1, 4, figsize=(20, 5))
for i in range(4):
  axs[i].imshow(image[i], cmap='gray')
  axs[i].set_title(f'Channel {i+1}')
  axs[i].axis('off')
plt.show()
rgb_image = image[:3, :, :].astype(float)  # Prendi solo i primi tre canali

rgb_image = rgb_image.transpose((1, 2, 0))  # Trasponi per ottenere la forma (height, width, channels)

plt.figure(figsize=(10, 10))
plt.imshow(rgb_image)
plt.title('RGB Image (Channels 1, 2, 3)')
plt.axis('off')
plt.show()

Funzione per calcolare media e deviazione standard

In [ ]:
def compute_mean_std(image_dir):
  num_channels = 4
  num_pixels = 0
  running_sum = np.zeros(num_channels)
  running_sum_squared = np.zeros(num_channels)

  for filename in os.listdir(image_dir):
    image_path = os.path.join(image_dir, filename)
    with rasterio.open(image_path) as src:
        image = src.read()
    flattened_image = image.reshape(-1, num_channels)
    running_sum += np.sum(flattened_image, axis=0)
    running_sum_squared += np.sum(flattened_image ** 2, axis=0)
    num_pixels += flattened_image.shape[0]

  mean = running_sum / num_pixels
  std = np.sqrt((running_sum_squared / num_pixels) - (mean ** 2))
  return mean, std

Calcolo media e deviazione standard

In [ ]:
mean, std = compute_mean_std(image_dir)
print("Mean: ", mean)
print("Std: ", std)